In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Budget Equation

From the previous section, we considered the following budget equations:
- **Full form**

$$
\frac{\partial q}{\partial t} + \nabla \cdot (\vec{u}q) = F
$$

- **Simplified component form** (neglecting sources/sinks and assuming Boussinesq fluids)

$$
\frac{\partial q}{\partial t} + u \frac{\partial q}{\partial x} + v \frac{\partial q}{\partial y} + \omega \frac{\partial q}{\partial p}= 0
$$


## Zonal Mean Operator

We now define the **zonal mean operator** as the average over the longitudinal ($x$) direction:

$$
\overline{(\cdot)} = \frac{1}{L_x}\int^{L_x}_{0}{(\cdot) \, dx}
$$

Applying this operator to the simplified budget equation yields:

$$
\frac{\partial \overline{q}}{\partial t} + \overline{u \frac{\partial q}{\partial x}} + \overline{v \frac{\partial q}{\partial y}} + \overline{\omega \frac{\partial q}{\partial p}} = 0
$$

## Reynolds Decomposition

To separate the mean and eddy components, we apply Reynolds decomposition:

$$
u = \overline{u} + u^{'}, \space v = \overline{v} + v^{'} ,\space \omega = \overline{\omega} + \omega^{'}, \space q = \overline{q} + q^{'}
$$

Substituting into the zonally averaged budget equation leads to terms involving combinations of mean and eddy components.
However, when applying the zonal mean operator, all mean–eddy cross terms vanish (proof is omitted).
As a result, only the **mean–mean** and **eddy–eddy** contributions remain:

$$
\frac{\partial \overline{q}}{\partial t} + (\overline{u^{'}\frac{\partial q^{'}}{\partial x}}) + (\overline{v}\frac{\partial \overline{q}}{\partial y} + \overline{v^{'}\frac{\partial q^{'}}{\partial y}}) + (\overline{\omega}\frac{\partial \overline{q}}{\partial p} + \overline{\omega^{'}\frac{\partial q^{'}}{\partial p}}) = 0
$$

## Demo

For simplicity, we focus only on eddy contributions in the $y$-direction, and neglect the mean terms:
$$
\frac{\partial \overline{q}}{\partial t} = - \overline{v^{'}\frac{\partial q^{'}}{\partial y}}
$$

We now express the eddies using Fourier series in the $x$-direction:
- $$v^{\prime} = \sum_{k}{\hat{v^{\prime}_{k}} e^{ikx}}$$
- $$\frac{\partial q^{'}}{\partial y} = \sum_{k}{\hat{(\frac{\partial q^{'}}{\partial y})_k} e^{ikx}}$$

Then the zonal mean of their product becomes:
$$
\overline{v^{'}\frac{\partial q^{'}}{\partial y}} = \sum_{k_1}{\sum_{k_2}{\hat{v^{\prime}_{k_1}} \hat{(\frac{\partial q^{'}}{\partial y})_{k_2}} \overline{e^{i(k_1+k_2)x}}}}
$$

From the properties of zonal mean operator:
$$\overline{e^{i(k_1 + k_2)x}} = 0 \quad \text{theoretically, for all } k_1, k_2 \text{ such that } k_1 + k_2 \neq 0.$$
$$\overline{e^{i(k_1 + k_2)x}} = 1 \quad \text{theoretically, for all } k_1, k_2 \text{ such that } k_1 + k_2 = 0.$$

Therefore, the only surviving terms are those with opposite wavenumbers:
$$
\frac{\partial \overline{q}}{\partial t} = - \overline{v^{'}\frac{\partial q^{'}}{\partial y}} = - \sum_{\substack{k_1, k_2 \\ k_1 + k_2 = 0}}{\hat{v^{\prime}_{k_1}} \hat{(\frac{\partial q^{'}}{\partial y})_{k_2}}}
$$

---
In this demo, I will verify that, under zonal averaging, for **real-value** functions
- Eddy contributions from different wavenumbers cancel out.
- Eddy contributions from same wavenumbers contribute to the budget.
- The total eddy contributions can thus be viewed as the sum of individual eddy–eddy interactions at each wavenumber.
---

In [ ]:
def display_eddy_structure(velocity_wavenumber : int = 1, quantity_gradient_wavenumber : int = 1) -> None:
    """
    Visualize the structure and spectral representation of eddy fluxes formed by
    the interaction between a velocity anomaly and a gradient quantity anomaly.

    Parameters:
        velocity_wavenumber (int): Wavenumber for the velocity perturbation v′. Must be a positive integer.
        quantity_gradient_wavenumber (int): Wavenumber for the quantity gradient ∂q′/∂y. Must be a positive integer.
        show_plot (bool): If True, displays the generated plots.

    Returns:
        None
    """
    if (
        not isinstance(velocity_wavenumber, int)
        or not isinstance(quantity_gradient_wavenumber, int)
        or velocity_wavenumber <= 0
        or quantity_gradient_wavenumber <= 0
    ):
        raise ValueError("velocity_wavenumber and quantity_gradient_wavenumber must be positive integers.")

    coordinate = np.linspace(0, 1, 1000, endpoint=False)
    freqencies = np.fft.fftfreq(len(coordinate), d=coordinate[1] - coordinate[0])

    v_prime = np.sin(2 * np.pi * velocity_wavenumber * coordinate)
    dqdy_prime = np.sin(2 * np.pi * quantity_gradient_wavenumber * coordinate)
    eddy_flux = v_prime * dqdy_prime

    v_spectrum = np.abs(np.fft.fft(v_prime, norm="ortho")) ** 2
    dqdy_spectrum = np.abs(np.fft.fft(dqdy_prime, norm="ortho")) ** 2
    eddy_spectrum = np.abs(np.fft.fft(eddy_flux, norm="ortho")) ** 2


    fig, axes = plt.subplots(
        nrows=2, ncols=2, figsize=(16, 9), dpi=160, sharex="col", sharey="col"
    )

    axes[0, 0].plot(coordinate, v_prime, label="$v′$", color="tab:red", linestyle='-', linewidth=2)
    axes[0, 0].plot(coordinate, dqdy_prime, label=r"$\frac{\partial q′}{\partial y}$", color="tab:blue", linestyle='--', linewidth=2)
    axes[0, 0].plot(coordinate, np.zeros_like(coordinate), color="black", linestyle='--', linewidth=1, zorder = -1)
    axes[0, 0].set_title(r"Individual Eddy Structure: $v′$ and $\frac{\partial q′}{\partial y}$")
    axes[0, 0].set_ylabel("Amplitude")
    axes[0, 0].legend(loc = 1)

    axes[0, 1].bar(freqencies, v_spectrum, label=r"$|v′|^2$", color="tab:red", width=0.4)
    axes[0, 1].bar(freqencies, dqdy_spectrum, label=r"$\left|\frac{\partial q′}{\partial y}\right|^2$", color="tab:blue", width=0.4, alpha = 0.5)
    axes[0, 1].set_title(r"Spectrum: $v′$ and $\frac{\partial q′}{\partial y}$")
    axes[0, 1].set_xlim(-10, 10)
    axes[0, 1].set_xticks(np.arange(-10, 11, 1))
    axes[0, 1].legend(loc = 1)

    axes[1, 0].plot(coordinate, eddy_flux, label=r"$v′ \frac{\partial q′}{\partial y}$", color="black", linewidth=2)
    axes[1, 0].fill_between(coordinate, 0, eddy_flux,
                         where=eddy_flux > 0,
                         facecolor='red',
                         interpolate=True,
                         alpha=0.5)
    axes[1, 0].fill_between(coordinate, 0, eddy_flux,
                         where=eddy_flux < 0,
                         facecolor='blue',
                         interpolate=True,
                         alpha=0.5)
    axes[1, 0].plot(coordinate, np.zeros_like(coordinate), color="black", linestyle='--', linewidth=1, zorder = -1)
    axes[1, 0].set_title(r"Eddy Interaction: $v′\frac{\partial q′}{\partial y}$")
    axes[1, 0].set_xlabel("Normalized Coordinate")
    axes[1, 0].set_ylabel("Amplitude")
    axes[1, 0].legend(loc = 1)

    axes[1, 1].bar(freqencies, eddy_spectrum, label=r"$|v′ \frac{\partial q′}{\partial y}|^2$", color="black", width=0.4)
    axes[1, 1].set_title(r"Spectrum: $v′\frac{\partial q′}{\partial y}$")
    axes[1, 1].set_xlabel("Wavenumber")
    axes[1, 1].set_xlim(-10, 10)
    axes[1, 1].set_xticks(np.arange(-10, 11, 1))
    axes[1, 1].legend(loc = 1)

    fig.suptitle(
        r"$v′$ wavenumber: " + f"{velocity_wavenumber}\n" +
        r"$\frac{{\partial q′}}{{\partial y}}$ wavenumber: " + f"{quantity_gradient_wavenumber}\n" +
        r"$\overline{v′\frac{\partial q′}{\partial y}}$: " +
        f"{np.round(np.mean(eddy_flux), 3)} (units)"
    )
    plt.tight_layout()
    plt.show()

    return None

def display_eddy_contribution_under_zonal_mean(
    velocity_wavenumber: int = 2,
    quantity_gradient_wavenumber: int = 2
) -> None:
    """
    Decompose eddy flux into contributions from individual wavenumbers
    by filtering velocity and gradient fields using Fourier methods.

    Parameters:
        velocity_wavenumber (int): Wavenumber of the velocity anomaly v′.
        quantity_gradient_wavenumber (int): Wavenumber of the quantity gradient anomaly ∂q′/∂y.

    Returns:
        None
    """
    def _compute_eddy_component(velocity: np.ndarray,quantity_gradient: np.ndarray,mask: np.ndarray):
        """
        Compute the contribution to the eddy flux from a specific wavenumber.

        Parameters:
            velocity (np.ndarray): velocity field (e.g., velocity anomaly).
            quantity_gradient (np.ndarray): quantity field (e.g., quantity gradient anomaly).
            mask (np.ndarray): Boolean mask for selecting a single wavenumber in frequency space.

        Returns:
            np.ndarray: Eddy flux component for the masked wavenumber.
        """
        p_tmp = np.fft.irfft(np.fft.rfft(velocity) * mask, n=velocity.size)
        q_tmp = np.fft.irfft(np.fft.rfft(quantity_gradient) * mask, n=quantity_gradient.size)
        eddy_component = p_tmp * q_tmp
        return eddy_component


    coordinate = np.linspace(0, 1, 1000, endpoint=False)
    
    v_prime = 10*np.sin(
        2 * np.pi * velocity_wavenumber * coordinate
    ) + 1e-2*np.random.randn(coordinate.size)
    dqdy_prime = 10*np.sin(
        2 * np.pi * quantity_gradient_wavenumber * coordinate
    ) + 1e-2*np.random.randn(coordinate.size)
    eddy_flux = v_prime * dqdy_prime

    total_eddy_contribution = np.zeros_like(coordinate, dtype=float)

    for wn in np.fft.rfftfreq(n=coordinate.size, d=1 / coordinate.size):
        wavenumber_mask = (
            np.fft.rfftfreq(n=coordinate.size, d=1 / coordinate.size) == wn
        )
        total_eddy_contribution += _compute_eddy_component(v_prime, dqdy_prime, wavenumber_mask)
    
    print(f"Sum of individual eddy contributions : {np.mean(total_eddy_contribution):.1f}")
    print(f"Total eddy contributions : {np.mean(eddy_flux):.1f}")
    return None


In [ ]:
display_eddy_structure(1, 3)
display_eddy_structure(2, 2)

In [ ]:
display_eddy_contribution_under_zonal_mean(1,3)
display_eddy_contribution_under_zonal_mean(2,2)